In [33]:
!pip install --upgrade langchain google-generativeai langchain-google-genai python-dotenv

  Using cached langchain_google_genai-2.0.11-py3-none-any.whl.metadata (3.6 kB)
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.


In [ ]:
import os

os.environ["GOOGLE_GENERATIVE_API_KEY"] = "your_API_KEY"

In [35]:
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_GENERATIVE_API_KEY = os.getenv("GOOGLE_GENERATIVE_API_KEY")

In [36]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Sử dụng eval một cách hạn chế built-in để tính toán biểu thức
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

In [37]:
from langchain.tools import tool

@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

In [38]:
from langchain_google_genai import GoogleGenerativeAI

# Khởi tạo mô hình với API key và model hiện hành ("gemini-pro" hay tên model tương ứng)
llm = GoogleGenerativeAI(model="gemini-pro", api_key=GOOGLE_GENERATIVE_API_KEY)

In [39]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# Khởi tạo mô hình Gemini-Pro với API Key
llm = GoogleGenerativeAI(model="gemini-1.5-pro-latest", api_key=GOOGLE_GENERATIVE_API_KEY)

# Tạo công cụ cho agent
tools = [calculator]

# Khởi tạo Agent với mô hình Gemini-Pro
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Chọn loại Agent phù hợp
    verbose=True
)

# Thực thi Agent
executor = agent

In [40]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Khởi tạo bộ nhớ hội thoại
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Tạo PromptTemplate cho hội thoại
prompt = PromptTemplate(
    input_variables=["chat_history", "input"],
    template="Lịch sử hội thoại: {chat_history}\nNgười dùng: {input}\nTrả lời:"
)

# Khởi tạo LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True  # Hiển thị log quá trình chạy
)


In [41]:
query = "What is 15 divided by 3?"
response = chain.run(query)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Lịch sử hội thoại: []
Người dùng: What is 15 divided by 3?
Trả lời:

> Finished chain.
15 divided by 3 is 5.



In [42]:
queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", chain.run(q))
    print("-" * 40)

Query: What is 25 multiplied by 4?


> Entering new LLMChain chain...
Prompt after formatting:
Lịch sử hội thoại: [HumanMessage(content='What is 15 divided by 3?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 divided by 3 is 5.\n', additional_kwargs={}, response_metadata={})]
Người dùng: What is 25 multiplied by 4?
Trả lời:

> Finished chain.
Response: 25 multiplied by 4 is 100.

----------------------------------------
Query: Now divide the result by 5.


> Entering new LLMChain chain...
Prompt after formatting:
Lịch sử hội thoại: [HumanMessage(content='What is 15 divided by 3?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 divided by 3 is 5.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 25 multiplied by 4?', additional_kwargs={}, response_metadata={}), AIMessage(content='25 multiplied by 4 is 100.\n', additional_kwargs={}, response_metadata={})]
Người dùng: Now divide the result by 5.
Trả lời:



> Finished chain.
Response: 100 divided by 5 is 20.

----------------------------------------
Query: Add 10 to that.


> Entering new LLMChain chain...
Prompt after formatting:
Lịch sử hội thoại: [HumanMessage(content='What is 15 divided by 3?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 divided by 3 is 5.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 25 multiplied by 4?', additional_kwargs={}, response_metadata={}), AIMessage(content='25 multiplied by 4 is 100.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Now divide the result by 5.', additional_kwargs={}, response_metadata={}), AIMessage(content='100 divided by 5 is 20.\n', additional_kwargs={}, response_metadata={})]
Người dùng: Add 10 to that.
Trả lời:



> Finished chain.
Response: 20 + 10 is 30.

----------------------------------------


In [43]:
query = "What is LangChain Function?"
response = chain.run(query)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Lịch sử hội thoại: [HumanMessage(content='What is 15 divided by 3?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 divided by 3 is 5.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is 25 multiplied by 4?', additional_kwargs={}, response_metadata={}), AIMessage(content='25 multiplied by 4 is 100.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Now divide the result by 5.', additional_kwargs={}, response_metadata={}), AIMessage(content='100 divided by 5 is 20.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Add 10 to that.', additional_kwargs={}, response_metadata={}), AIMessage(content='20 + 10 is 30.\n', additional_kwargs={}, response_metadata={})]
Người dùng: What is LangChain Function?
Trả lời:

> Finished chain.
LangChain is a framework designed to simplify the development of applications using large language models (LLMs). It pr